In [1]:
from io import BytesIO
import base64
from PIL import Image
from math import ceil
import requests
import cv2
from multiprocessing import Pool
from pathlib import Path
import time
import torch
from tqdm import tqdm


In [16]:
api = 'http://localhost:3000/api/'
secret = 'seva_is'

In [17]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=Path('./best.pt'))
cameras = list((Path() / '..' / 'public' / 'dataset').glob('*'))
frames = []
for camera in cameras:
    frames += list(camera.glob('*.jpg'))
frames = sorted(frames, key=lambda frame: frame.name)

Using cache found in /home/sevskii/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-4 torch 1.10.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070 Ti, 8103MiB)

Fusing layers... 
Model Summary: 378 layers, 35335500 parameters, 0 gradients, 49.2 GFLOPs
Adding AutoShape... 


In [21]:
camera_usage = {}
batch = []
raw_batch = []
for frame in tqdm(frames):
    camera_id = frame.parent.name
    if camera_id in camera_usage:
        preds = model(batch)
        batch_result = []

        for j in range(len(preds)):
            xywhn = preds.xywhn[j].cpu()
            labels = xywhn.T[-1].to(int)
            scores = xywhn.T[-2]
            boxes = xywhn.T[0:4].T
            boxes[:, 0] -= boxes[:, 2] / 2
            boxes[:, 1] -= boxes[:, 3] / 2

            curr_frame = raw_batch[j]
            
            timestamp_int = int(time.mktime(time.strptime(curr_frame.name.replace("_AM", "").replace("_PM", ""), "%d-%m-%Y_%H-%M-%S.jpg")))
            if "_PM" in curr_frame.name:
                timestamp_int += 43200 
            
            batch_result.append({
             'camera_id': int(curr_frame.parent.name),
             'timestamp': curr_frame.name,
             'timestamp_int': timestamp_int,
             'labels': labels.tolist(),
             'scores': scores.tolist(),
             'boxes': boxes.tolist()
            })
        requests.post(api + 'save_frames', json={'secret': secret, 'payload': batch_result, 'buildHeatmap': True})
        #requests.get(api + 'calc_heatmap', {'secret': secret})
        camera_usage = {}
        batch = []
        raw_batch = []
    camera_usage[camera_id] = 1
    batch.append(frame)
    raw_batch.append(frame)
    

100%|██████████| 13110/13110 [10:36<00:00, 20.59it/s]
